In [ ]:
!pip install dash dash-bootstrap-components pandas scikit-learn
!pip install pyngrok
from pyngrok import ngrok
!pip install dash dash-bootstrap-components pandas scikit-learn pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 12.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.5/222.5 kB 7.3 MB/s eta 0:00:00


In [ ]:
!pip install dash dash-bootstrap-components pandas scikit-learn flask-ngrok


In [ ]:
import dash
from dash import dcc, html
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output, State
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import plotly.express as px
import plotly.graph_objects as go
from flask_ngrok import run_with_ngrok

# Cargar los datos
file_path = '/content/6 class csv.csv'
data = pd.read_csv(file_path)

# Preparar los datos para el entrenamiento
X = data[['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)']]
y = data['Star type']

# Dividir los datos en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Estandarizar las características
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Entrenar el modelo SVM
svm_model = SVC(kernel='linear', random_state=42)
svm_model.fit(X_train_scaled, y_train)

# Evaluar el modelo
y_pred_train = svm_model.predict(X_train_scaled)
y_pred_test = svm_model.predict(X_test_scaled)
train_accuracy = accuracy_score(y_train, y_pred_train)
test_accuracy = accuracy_score(y_test, y_pred_test)

# Crear la aplicación Dash
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
run_with_ngrok(app)  # Inicia ngrok cuando se ejecute el servidor

app.layout = dbc.Container([
    dbc.Tabs([
        dbc.Tab(label='Clasificación de Estrellas', tab_id='tab-1'),
        dbc.Tab(label='Información del Modelo', tab_id='tab-2')
    ], id='tabs', active_tab='tab-1'),
    html.Div(id='content')
])

@app.callback(
    Output('content', 'children'),
    Input('tabs', 'active_tab')
)
def render_content(tab):
    if tab == 'tab-1':
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H1("Clasificación de Estrellas", className='text-center my-4'),
                    dbc.Card(
                        dbc.CardBody([
                            html.Label("Temperatura (K):"),
                            dcc.Input(id='input-temp', type='number', value=5000, step=100, min=0, max=40000, className='form-control mb-2'),
                            html.Br(),
                            html.Label("Luminosidad (L/Lo):"),
                            dcc.Input(id='input-lum', type='number', value=1, step=0.1, className='form-control mb-2'),
                            html.Label("Radio (R/Ro):"),
                            dcc.Input(id='input-radius', type='number', value=1, step=0.1, className='form-control mb-2'),
                            html.Label("Magnitud Absoluta (Mv):"),
                            dcc.Input(id='input-mag', type='number', value=5, step=0.1, className='form-control mb-2'),
                            html.Br(),
                            dbc.Button("Clasificar", id='classify-button', color='primary', className='mt-2'),
                            html.Hr(),
                            html.Div(id='output-class', className='text-center mt-4')
                        ]),
                    )
                ], width=4),
                dbc.Col([
                    html.H3("Gráfico de Dispersión de Estrellas", className='text-center my-4'),
                    dcc.Graph(id='star-scatter', figure={})
                ], width=8)
            ], className='mt-4')
        ])
    elif tab == 'tab-2':
        return dbc.Container([
            dbc.Row([
                dbc.Col([
                    html.H5("Modelo Utilizado: Support Vector Machine (SVM)"),
                    html.P("El modelo SVM con kernel lineal se utiliza para clasificación."),
                    html.P("El modelo fue elegido por su efectividad en problemas de clasificación con datos de alta dimensionalidad."),
                    html.P(f"Precisión del conjunto de entrenamiento: {train_accuracy:.2%}"),
                    html.P(f"Precisión del conjunto de prueba: {test_accuracy:.2%}")
                ], width=12)
            ], className='mt-4'),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(
                        id='accuracy-graph',
                        figure=go.Figure(
                            data=[go.Bar(x=['Train Accuracy', 'Test Accuracy'], y=[train_accuracy, test_accuracy], text=[f'{train_accuracy:.2%}', f'{test_accuracy:.2%}'], textposition='auto')],
                            layout_title_text='Precisión del Modelo'
                        )
                    )
                ], width=12)
            ], className='mt-4')
        ])

@app.callback(
    Output('output-class', 'children'),
    Output('star-scatter', 'figure'),
    Input('classify-button', 'n_clicks'),
    State('input-temp', 'value'),
    State('input-lum', 'value'),
    State('input-radius', 'value'),
    State('input-mag', 'value')
)
def classify_star(n_clicks, temp, lum, radius, mag):
    if n_clicks is None:
        return "", {}

    # Preprocesar los valores de entrada
    input_data = pd.DataFrame([[temp, lum, radius, mag]], columns=['Temperature (K)', 'Luminosity(L/Lo)', 'Radius(R/Ro)', 'Absolute magnitude(Mv)'])
    input_scaled = scaler.transform(input_data)

    # Realizar la predicción
    star_type = svm_model.predict(input_scaled)[0]

    # Crear la anotación para la estrella consultada
    annotation = {
        'x': temp,
        'y': lum,
        'xref': 'x',
        'yref': 'y',
        'text': f"Estrella Consultada<br>Tipo: {star_type}",
        'showarrow': True,
        'arrowhead': 2,
        'ax': 0,
        'ay': -40
    }

    # Actualizar el gráfico de dispersión
    fig = px.scatter(data, x='Temperature (K)', y='Luminosity(L/Lo)', color='Star type',
                     title='Clasificación de Estrellas en la Secuencia Principal',
                     labels={'Temperature (K)': 'Temperatura (K)', 'Luminosity(L/Lo)': 'Luminosidad (L/Lo)'},
                     hover_data=['Radius(R/Ro)', 'Absolute magnitude(Mv)', 'Star color', 'Spectral Class'])
    fig.update_layout(
        transition_duration=500,
        annotations=[annotation],
        xaxis_autorange='reversed',  # Invertir el eje x
        yaxis_type='log'  # Escala logarítmica para el eje y
    )

    return dbc.Alert(f"Tipo de estrella predicho: {star_type}", color='info'), fig

# Índice de tipos de estrellas
index_content = dbc.Container([
    html.H5("Índice de Tipos de Estrellas"),
    html.P("Brown Dwarf -> Star Type = 0"),
    html.P("Red Dwarf -> Star Type = 1"),
    html.P("White Dwarf -> Star Type = 2"),
    html.P("Main Sequence -> Star Type = 3"),
    html.P("Supergiant -> Star Type = 4"),
    html.P("Hypergiant -> Star Type = 5")
], className='mt-4')

# Agregar el índice al layout de la aplicación
app.layout.children.append(index_content)

# Ejecutar la aplicación
app.run_server()


SyntaxError: invalid syntax (<ipython-input-5-031a84d26ad0>, line 16)